In [6]:
from google import genai
from google.genai.types import GenerateContentConfig
import os, json

api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client(api_key=api_key)

tools = [
    {"url_context": {}},
    {"google_search": {}}
]



url = "https://www.youtube.com/watch?v=RgKAFK5djSk"
language = "Vietnmese"



prompt = f"""
ROLE
You are an assistant that summarizes YouTube videos.

TASK
Given a YouTube video URL, analyze the video content and generate a structured summary. You may infer information from the video, including title, description, chapters, subtitles, and spoken content if available.

Then do the following:
- Return ONLY valid JSON
- Do NOT include explanations or extra text
- You can process this task by every language but OUPUTPUT LANGUAGE MUST{language}
- Use Markdown-compatible bullet points ("-")
- Ensure the structure strictly follows the schema below
- Limit total output to a maximum of 30,000 tokens
- If the video is very long, limit the number of segments so the output does not exceed this limit
- Only provide a concise and representative selection of the most important insights and summary segments as needed

Youtube Video URL:
{url}

JSON SCHEMA
{{
  "insights": [
    {{
      "insight": "Insight 1",
      "sub-insights": [
        "Insight point 1",
        "Insight point 2"
      ]
    }}
  ],
  "summary": {{
    "tldr": "One line summary of the video",
    "timestamps": [
      {{
        "time": "MM:SS",
        "start_seconds": 0,
        "text": "Summary segment 1"
      }}
    ]
  }}
}}

FINAL INSTRUCTION
Generate the JSON now.
"""

import json
import re

response = client.models.generate_content(
    model="models/gemini-2.5-flash-lite",
    contents=prompt,
    config=GenerateContentConfig(
        tools=tools
    )
)

# --- Extract text from parts ---
print(response)




sdk_http_response=HttpResponse(
  headers=<dict len=11>
) candidates=[Candidate(
  content=Content(
    parts=[
      Part(
        text="""```json
{
  "insights": [
    {
      "insight": "Bài hát \"See You Again\" là một bản nhạc tưởng nhớ dành cho Paul Walker.",
      "sub-insights": [
        "Bài hát được phát hành trong Phần phim \"Furious 7\" và là một phần của nhạc phim.",
        "Video âm nhạc có những cảnh quay tri ân đến Paul Walker."
      ]
    },
    {
      "insight": "Nghệ sĩ và sự hợp tác.",
      "sub-insights": [
        "Bài hát được thể hiện bởi Wiz Khalifa và Charlie Puth.",
        "Charlie Puth đã sáng tác và sản xuất bài hát này."
      ]
    },
    {
      "insight": "Thành công và tầm ảnh hưởng của bài hát.",
      "sub-insights": [
        "Bài hát đã đạt được lượng xem khổng lồ trên YouTube.",
        "Nó trở thành một bản hit toàn cầu và được yêu thích rộng rãi."
      ]
    }
  ],
  "summary": {
    "tldr": "See You Again là một bài hát tưởng nhớ đầy cảm